# pycisTopic analysis

Full dataset, using consensus peak regions.

In [1]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import pickle
import pandas as pd

In [4]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged'
os.chdir( wdir )

In [5]:
# create output directory:
f_final_dir = os.path.join(wdir, 'downstream_analysis')
if not os.path.exists(f_final_dir):
    os.makedirs(f_final_dir)

In [6]:
import copy

## Load the cisTopic objects

In [7]:
f_cto_dir = 'pycistopic_consensus_peaks/cistopic_objs__mergedconsensus/'

cistopic_obj_dict = {}
for key in ['libds_merged']:
    f_cto = os.path.join(wdir, f_cto_dir, key + '__cistopic_obj_mergedconsensus_metadata_annotated_models_umap_tsne.pkl')
    if(os.path.isfile(f_cto)):
        with open(f_cto, 'rb') as f:
            cistopic_obj_dict[key] = pickle.load(f)
        print(f"Loaded filtered cistopic object {key}")
    else:
        print(f"file {f_cto} doesn't exist")

Loaded filtered cistopic object libds_merged


In [8]:
from collections import OrderedDict

In [9]:
cistopic_obj_dict['libds_merged'].cell_data['sample_id'].unique()

array(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1',
       'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2',
       'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'],
      dtype=object)

In [10]:
# display names of each sample. NOTE: the samples will be dislpayed in the order of this dict!
alias_dict = OrderedDict({
    "Broad_1": "BioRad ATAC 1",
    "Broad_2": "BioRad ATAC 2",
    "Stanford_1": "10x ATAC A1",
    "Stanford_2": "10x ATAC A2",
    "VIB_1": "10x ATAC B1",
    "VIB_2": "10x ATAC B2",
    "CNAG_1": "10x ATAC C1",
    "CNAG_2": "10x ATAC C2",
    "Broad_mito_1": "10x mtATAC 1",
    "Broad_mito_2": "10x mtATAC 2",
    "Sanger_1": "10x Multiome 1",
    "Sanger_2": "10x Multiome 2",
    "VIB_Hydrop_1": "Hydrop ATAC 1",
    "VIB_Hydrop_2": "Hydrop ATAC 2",
    "s3atac": "s3 ATAC",
    "merged": "Merged"
})

In [11]:
cistopic_obj_dict['libds_merged'].cell_data['alias'] = [alias_dict[x] for x in cistopic_obj_dict['libds_merged'].cell_data['sample_id']]

In [12]:
cistopic_obj_dict['libds_merged'].cell_data['sample_id'].unique()

array(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1',
       'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2',
       'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'],
      dtype=object)

In [13]:
for key in cistopic_obj_dict['libds_merged'].cell_data['sample_id'].unique():
    print(alias_dict[key])

BioRad ATAC 1
BioRad ATAC 2
10x mtATAC 1
10x mtATAC 2
10x ATAC C1
10x ATAC C2
10x Multiome 1
10x Multiome 2
10x ATAC A1
10x ATAC A2
10x ATAC B1
10x ATAC B2
Hydrop ATAC 1
Hydrop ATAC 2
s3 ATAC


#### Save/load

In [14]:
f_out = os.path.join(f_final_dir, 'region_bin_topics.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        region_bin_topics_dict = pickle.load(f)

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/region_bin_topics.pkl


In [15]:
f_out = os.path.join(f_final_dir, 'binarized_cell_topic.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        binarized_cell_topic_dict = pickle.load(f)

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/binarized_cell_topic.pkl


In [16]:
f_out = os.path.join(f_final_dir, 'imputed_acc_obj.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        imputed_acc_obj_dict = pickle.load(f)

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/imputed_acc_obj.pkl


In [17]:
f_out = os.path.join(f_final_dir, 'markers_dict.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        markers_dict_dict = pickle.load(f)

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/markers_dict.pkl


## Gene activity

In [18]:
import pyranges as pr
import requests
import pybiomart as pbm

### Infer gene activity

In [19]:
f_out = os.path.join(f_final_dir, 'gene_act_dict.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        gene_act_dict = pickle.load(f)
else:
    from pycisTopic.gene_activity import get_gene_activity
    from pycisTopic.diff_features import find_diff_features
    # For human
    dataset = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')
    annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position', 'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])
    annot['Chromosome/scaffold name'] = 'chr' + annot['Chromosome/scaffold name'].astype(str)
    annot.columns=['Chromosome', 'Start', 'End', 'Strand', 'Gene','Transcription_Start_Site', 'Transcript_type']
    annot = annot[annot.Transcript_type == 'protein_coding']
    annot.Strand[annot.Strand == 1] = '+'
    annot.Strand[annot.Strand == -1] = '-'
    pr_annotation = pr.PyRanges(annot.dropna(axis = 0))
    pr_annotation

    # get chromosome sizes (hg38)
    target_url = 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
    chromsizes = pd.read_csv(target_url, sep='\t', header=None)
    chromsizes.columns = ['Chromosome', 'End']
    chromsizes['Start'] = [0]*chromsizes.shape[0]
    chromsizes = chromsizes.loc[:,['Chromosome', 'Start', 'End']]
    chromsizes = pr.PyRanges(chromsizes)
    chromsizes

    gene_act_dict = {}
    for key in cistopic_obj_dict.keys():
        print(key)
        gene_act, weights = get_gene_activity(
            imputed_acc_obj_dict[key], # Region-cell probabilities
            pr_annotation, # Gene annotation
            chromsizes, # Chromosome size
            use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
            upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it 
                                     #these bp will be taken (1kbp here)
            downstream=[1000,100000], # Search space downstream
            distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
            decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
            extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for 
                                             #this weight)
            extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
            gene_size_weight=False, # Whether to add a weights based on the length of the gene
            gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                                             #in the genome
            remove_promoters=False, # Whether to remove promoters when computing gene activity scores
            average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene 
                                 # activity score
            scale_factor=1, # Value to multiply for the final gene activity matrix
            extend_tss=[10,10], # Space to consider a promoter
            gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
            return_weights= True, # Whether to return the final weights
            project='Gene_activity') # Project name for the gene activity object
        gene_act_dict[key] = copy.copy(gene_act)

    with open(f_out, 'wb') as f:
        pickle.dump(gene_act_dict, f)
    print(f'saved {f_out}')

Loading /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/gene_act_dict.pkl


In [20]:
cistopic_obj_dict['libds_merged'].projections

{'cell': {'UMAP':                                                        UMAP_1     UMAP_2
  CCTCCTTCTTCATCCAGAGAG_TAAGAGGGTGGCGCCTTGCGA_CGG...   5.778868  11.895809
  CCTTAGGACGAGAATTATCAT_CCGCGATACCTACCAGATAGG-Bro...  -0.684854  19.437815
  ATGAATAGTGCATTGCAGTGT-Broad_1                        2.187975  14.684908
  TGTTTAGATAGGCATAAGGTA-Broad_1                        6.169397  11.672524
  TTACAGAGGTGTTTCCAAGCT_GGACGACAGTTTCTCTAGAGC-Bro...   5.618959  11.884871
  ...                                                       ...        ...
  GGTTAGTTGGTATTGCAGCTCGGACAAC-s3atac                -10.563517   3.115543
  GATTCGGTCAGTTCTCTCCTTGACGAAT-s3atac                -10.507968   3.265422
  TGCGGCCTGGTCTCATTGCCCGGAACTG-s3atac                 -7.065394  -2.570484
  GAAGAGTATTTCTCCTCCTGGTGTCGGA-s3atac                -10.610051   2.621628
  TGCGGCCTGGAATGATGCTCATTGTGAA-s3atac                 -7.511748  -3.023273
  
  [45235 rows x 2 columns],
  'tSNE':                                            

## Export to loom

In [21]:
f_gene_loom_dir = os.path.join(f_final_dir, 'gene_act_loom')
if not os.path.exists(f_gene_loom_dir):
    os.makedirs(f_gene_loom_dir)

In [22]:
from pycisTopic.loom import (
    export_region_accessibility_to_loom,
    export_gene_activity_to_loom
)

### Gene activity

In [23]:
cistopic_obj_dict['libds_merged'].cell_data.columns

Index(['Unique_nr_frag_in_regions', 'sample_id', 'cisTopic_nr_frag',
       'seurat_cell_type', 'pycisTopic_leiden_10_1.0', 'fmx_sample',
       'Log_unique_nr_frag', 'pycisTopic_leiden_10_0.8', 'Unique_nr_frag',
       'consensus_cell_type', 'Total_nr_frag_in_regions', 'Dupl_rate',
       'cisTopic_nr_acc', 'TSS_enrichment', 'pycisTopic_leiden_10_0.6', 'FRIP',
       'cisTopic_log_nr_frag', 'Dupl_nr_frag', 'Total_nr_frag',
       'Log_total_nr_frag', 'cisTopic_log_nr_acc', 'barcode',
       'pycisTopic_leiden_10_1.2', 'UMAP_harmony_1', 'UMAP_harmony_2',
       'tSNE_harmony_1', 'tSNE_harmony_2', 'alias'],
      dtype='object')

In [24]:
from ctxcore.genesig import Regulon

# generate a dummy regulon (required for export_gene_activity_to_loom):
phreg = Regulon(
        name='placeholder regulon',
        gene2weight={'phreg': 1.0},
        transcription_factor="phreg",
        gene2occurrence={"phreg": 1},
    )

In [25]:
for key in cistopic_obj_dict.keys():
    print(key)
    s = 'Merged' if key=='libds_merged' else alias_dict[key]
    f_out = os.path.join(f_gene_loom_dir, s + '__libds_gene_activity.loom')
    if os.path.exists(f_out):
        print(f"Skipping {f_out}: already exists.")
        continue
        
    export_gene_activity_to_loom(
        gene_activity_matrix = gene_act_dict[key],
        cistopic_obj = cistopic_obj_dict[key], 
        regulons = [phreg],
        # selected_cells = [ x.split('-')[0] + '-' + x.split('-')[1]  for x in cistopic_obj_dict[key].cell_names ], # this leaves a cell barcode of the format type 'TGCATGTCGCCGTTCCAAGA-21'
        # selected_cells = cistopic_obj_dict[key].projections['cell']['UMAP'].index.tolist(), # cflerin original 
        out_fname = f_out,
        cluster_annotation = ['consensus_cell_type', 'alias', 'fmx_sample'],
        cluster_markers = {'consensus_cell_type': markers_dict_dict[key], 'alias':{}, 'fmx_sample':{}},
        tree_structure = ('scATAC-seq_Benchmark', 'ATAC_library_downsampled', 'Gene_activity'),
        title = 'Gene activity from library-downsampled and sample-merged dataset',
        nomenclature = "hg38"
    )

libds_merged
Skipping /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/gene_act_loom/Merged__libds_gene_activity.loom: already exists.


### Region accessibility

In [26]:
f_region_loom_dir = os.path.join(f_final_dir, 'region_acc_loom')
if not os.path.exists(f_region_loom_dir):
    os.makedirs(f_region_loom_dir)

In [ ]:
for key in cistopic_obj_dict.keys():
    print(key)
    s = 'Merged' if key=='libds_merged' else alias_dict[key]
    f_out = os.path.join(f_region_loom_dir, s + '__libds_region_accessibility.loom')

    if os.path.exists(f_out):
        print(f"Skipping {f_out}: already exists.")
        continue
    # Subset regions, we will use only regions in topics and DARs here to make it faster
    regions_in_topics = list(set(sum([region_bin_topics_dict[key][i].index.tolist() for i in region_bin_topics_dict[key].keys()],[])))
    regions_in_DARs = list(set(sum([markers_dict_dict[key][i].index.tolist() for i in markers_dict_dict[key].keys()],[])))
    # make sure we only take regions that actually exist in the accessibility matrix:
    selected_regions = list(set(regions_in_topics + regions_in_DARs).intersection(set(imputed_acc_obj_dict[key].feature_names)))

    # Export to loom
    export_region_accessibility_to_loom(
        accessibility_matrix = imputed_acc_obj_dict[key],
        cistopic_obj = cistopic_obj_dict[key], 
        binarized_topic_region = region_bin_topics_dict[key],
        binarized_cell_topic = binarized_cell_topic_dict[key],
        out_fname = f_out,
        selected_regions = selected_regions ,
        # selected_cells = [ x.split('-')[0] + '-' + x.split('-')[1]  for x in cistopic_obj_dict[key].cell_names ], # this leaves a cell barcode of the format type 'TGCATGTCGCCGTTCCAAGA-21'
        # selected_cells = cistopic_obj_dict[key].projections['cell']['UMAP'].index.tolist(), # cflerin original
        cluster_annotation = ['consensus_cell_type', 'alias', 'fmx_sample'],
        cluster_markers = {'consensus_cell_type': markers_dict_dict[key], 'alias':{}, 'fmx_sample':{}},
        tree_structure = ('scATAC', 'Region_accessibility'),
        title = f'Region accessibility from sample-merged dataset',
        nomenclature = "hg38"
    )

# Caution:
The export loom function contains a bug which adds the barcodes as a clustering, causing scope to crash on loading. We fix this below:

# open loom

In [28]:
import loompy as lp
import json
import copy
import glob
from collections import OrderedDict

exclude 'barcode' annotation

In [29]:
filenames = sorted(glob.glob(f_gene_loom_dir+'/*loom'))
samples = [item.replace("__libds_gene_activity.loom", "") for item in filenames]
samples = [item.replace("downstream_analysis/gene_act_loom/", "") for item in samples]
loom_dict = {samples[i]: filenames[i] for i in range(len(samples))}
loom_dict = OrderedDict(loom_dict)

In [30]:
loom_dict

OrderedDict([('/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/Merged',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/gene_act_loom/Merged__libds_gene_activity.loom')])

In [31]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'barcode':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\tbarcode index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno barcode annotation detected')
        loom.close()

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/gene_act_loom/Merged__libds_gene_activity.loom
	no barcode annotation detected


# remove the pesky sample_id from the non-merged looms

In [32]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'sample_id':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\sample_id index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno sample_id detected')
        loom.close()
    print('\n')

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/gene_act_loom/Merged__libds_gene_activity.loom
	no sample_id detected




# open loom

In [33]:
import loompy as lp
import json
import copy
import glob
from collections import OrderedDict

exclude 'barcode' annotation

In [34]:
filenames = sorted(glob.glob(f_region_loom_dir+'/*loom'))
samples = [item.replace("__libds_region_accessibility.loom", "") for item in filenames]
samples = [item.replace("downstream_analysis/region_acc_loom/", "") for item in samples]
loom_dict = {samples[i]: filenames[i] for i in range(len(samples))}
loom_dict = OrderedDict(loom_dict)

In [35]:
loom_dict

OrderedDict([('/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/Merged',
              '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/region_acc_loom/Merged__libds_region_accessibility.loom')])

In [36]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'barcode':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\tbarcode index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno barcode annotation detected')
        loom.close()

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/region_acc_loom/Merged__libds_region_accessibility.loom
	barcode index found: 4
	fixed /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/Merged


# remove the pesky sample_id from the non-merged looms

In [37]:
for sample in loom_dict.keys():
    file = loom_dict[sample]
    print(f'opening {file}')
    loom = lp.connect(file, mode='r+', validate=False)

    metadata = json.loads(loom.attrs['MetaData'])
    new_metadata = copy.deepcopy(metadata)
    for item in new_metadata['annotations']:
        if item['name'] == 'sample_id':
            barcode_index = new_metadata['annotations'].index(item)

    if 'barcode_index' in locals():
        print(f'\sample_id index found: {barcode_index}')
        new_metadata['annotations'].pop(barcode_index)
        del barcode_index

        new_metadata_json = json.dumps(new_metadata)
        loom.attrs['MetaData'] = new_metadata_json
        loom.close()
        print(f'\tfixed {sample}')

    else:
        print('\tno sample_id detected')
        loom.close()
    print('\n')

opening /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/downstream_analysis/region_acc_loom/Merged__libds_region_accessibility.loom
\sample_id index found: 0
	fixed /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/5_downsampled_data_merged/Merged


